# **IMPORT LIBRERIE**

In [ ]:
import tensorflow as tf 
import numpy as np
import csv


# **APRE FILE CSV CON LE LABEL**

In [ ]:
csv_file = open('/home/paolo/ComputerSync/Università/ProgettoAgeEstimation/AgeEstimation/data/train.age_detected.csv')
reader = csv.reader(csv_file, delimiter=',', quotechar='|')
rows = list(reader)


# **LEGGE IL FILE**

In [ ]:
y = []
for row in rows:
  value = round(float(row[1]))
  y.append(value)
  

In [ ]:
print(np.unique(y))
print(np.bincount(y))


# **CREA ARRAY CON NUMERO DI CAMPIONI PER OGNI CLASSE**

In [ ]:
bincount = np.bincount(y)
bincount = np.clip(bincount,0,1000)
length=82000//0.85
rest = length-np.sum(bincount)

while True:
    if rest == 0:
        break
    for i in range(bincount.shape[0]-1,0,-1):
        if rest == 0:
            break
        if bincount[i] >= 1000:
            bincount[i]+=1
            rest -= 1


# **SELEZIONA LE LABELS DAL DATASET ORIGINALE**

In [ ]:
from random import shuffle

for i in range(5):
    shuffle(rows)

new_labels_filename = []
new_labels = []
for row in rows:
    value = round(float(row[1]))
    if bincount[value] > 0:
        new_labels_filename.append(row[0])
        new_labels.append(value) 
        bincount[value] -= 1


# **GRAFICO BILANCIAMENTO CLASSI DOPO PREPROCESSING**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(figsize =(20, 15)) 
ax.hist(new_labels, bins = np.arange(84)) 
plt.show()
print("Media età\t\t", y.mean())
print("Deviazione standard età\t", np.std(y))
print("Età minima\t\t", y.min())
print("Età massima\t\t", y.max())


# **SPLIT TRAINING E VALIDATION SET**

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_valid,y_train,y_valid = train_test_split(new_labels_filename,new_labels,test_size=0.15,shuffle=True)

# **SALVA TRAIN E VALIDATION LABELS IN DUE FILE SEPARATI**

In [ ]:
train_labels = []
for i in range(len(x_train)):
    train_labels.append([x_train[i],y_train[i]])

with open('/home/paolo/ComputerSync/Università/ProgettoAgeEstimation/AgeEstimation/data/train_labels.csv', 'w') as f: 
    write = csv.writer(f) 
    write.writerows(train_labels)

valid_labels = []
for i in range(len(x_valid)):
    valid_labels.append([x_valid[i],y_valid[i]])

with open('/home/paolo/ComputerSync/Università/ProgettoAgeEstimation/AgeEstimation/data/valid_labels.csv', 'w') as f: 
    write = csv.writer(f) 
    write.writerows(valid_labels)

# **CALCOLA I PESI PER OGNI CLASSE E SALVA SU FILE**

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights = dict(enumerate(class_weights))

w = []
for key in class_weights:
    w.append([key,class_weights[key]])
    print(key,class_weights[key])
with open('/home/paolo/ComputerSync/Università/ProgettoAgeEstimation/AgeEstimation/data/weights.csv', 'w') as f:
    write = csv.writer(f)
    write.writerows(w) 